# 04 - Palm Interview Response Evaluation
`Author: Abdlazeez Jimoh`

In [1]:
import importlib
import json
from pprint import pprint

from typing_extensions import Literal, TypedDict

In [2]:
PALM_API_KEY = "YOUR_API_KEY"

In [3]:
class Question(TypedDict):
    question: str
    type: Literal["personal", "role-specific", "behavioral", "situational"]


class Evaluation(TypedDict):
    evaluation: Literal["good", "average", "bad"]
    feedback: str | None
    reason: str | None
    samples: list[str] | None

In [4]:
class PalmResponseEvaluationAgent:
    def __init__(self):
        self.client = importlib.import_module("google.generativeai")
        self.client.configure(api_key=PALM_API_KEY)
        self.system_prompt = """You are an interviewer evaluating a candidate's response to an interview question. Your task is to:
- Evaluate the candidate's response on the scale of "good", "average", and "bad".
- Provide a reason for why it's categorized as good, average, or bad.
- Offer constructive feedback or suggestions for improvement.
- Provide 2 samples of good responses.

You will be provided with an interview question and a candidate response.

Evaluate and provide output in the following JSON format:
{{
    "evaluation": "good, average, or bad",
    "reason": "Reason why it's good, average, or bad",
    "feedback": "Feedback or suggestions for improvement",
    "samples": [
        "Good response 1", 
        "Good response 2"
    ]
}}


===
QUESTION:
{question}

RESPONSE: 
{response}"""

    def __call__(self, question: str, response: str) -> Evaluation | None:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            Evaluation | None: The evaluation of the candidate's response or None if an error occurred.
        """

        # Generate questions
        evaluation = self._generate(question, response)

        return evaluation

    def run(self, question: str, response: str) -> Evaluation | None:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            Evaluation | None: The evaluation of the candidate's response or None if an error occurred.
        """

        # Generate questions
        evaluation = self._generate(question, response)

        return evaluation

    def _generate(self, question: str, response: str) -> Evaluation | None:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            Evaluation | None: The evaluation of the candidate's response or None if an error occurred.
        """

        try:
            output = self.client.generate_text(
                model="models/text-bison-001",
                prompt=self.system_prompt.format(question=question, response=response),
                temperature=1,
                max_output_tokens=1024,
            )

            evaluations = json.loads(output.result)

            return evaluations
        except Exception as e:
            return None

In [5]:
response_evaluator = PalmResponseEvaluationAgent()
evaluation = response_evaluator.run(
    "What motivated you to pursue a career in software engineering at a startup in San Francisco?",
    "I've always been interested in technology and startups. I've been working in the tech industry for the past 5 years and I've always wanted to work at a startup. I'm really excited about the opportunity to work at a startup in San Francisco."
)

pprint(evaluation)

/home/abdulazeezoj/Devspace/Work/omdena/local-chapter/india-chapter/chatbot-interview-prep/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'evaluation': 'good',
 'feedback': 'None',
 'reason': 'The response is well-written and provides a clear and concise '
           "answer to the question. It also demonstrates the candidate's "
           'interest in the tech industry and startups.',
 'samples': ["I've always been interested in technology and startups. I've "
             "been working in the tech industry for the past 5 years and I've "
             "always wanted to work at a startup. I'm passionate about "
             "building new products and solving real-world problems. I'm also "
             'excited about the opportunity to work at a startup in San '
             "Francisco because it's a great place to be for the tech "
             'industry.',
             "I'm a software engineer with a passion for building innovative "
             "products. I've been working in the tech industry for the past 5 "
             "years and I've always wanted to work at a startup. I'm excited "
             'about the opp

In [6]:
evaluation = response_evaluator.run(
    "How do you prioritize tasks when leading a team of data scientists and engineers on multiple projects?",
    "I prioritize tasks by importance and urgency. I also make sure to communicate with my team about what needs to be done and when it needs to be done. I also make sure to communicate with my team about what needs to be done and when it needs to be done.",
)

pprint(evaluation)

{'evaluation': 'good',
 'feedback': 'None.',
 'reason': 'The candidate provides a clear and concise answer to the question. '
           'They also mention the importance of communicating with the team '
           'about what needs to be done and when it needs to be done.',
 'samples': ['I prioritize tasks by importance and urgency. I also make sure '
             'to delegate tasks to the most qualified members of my team.',
             "I use a Kanban board to visualize the progress of my team's work "
             'and to ensure that we are meeting our deadlines.']}
